## Privacy Rights Clearinghouse Data Breaches Data Wrangling

### Loading data

In [1]:
#import libraries
import pandas as pd
import numpy as np
import datetime as dt
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib

In [2]:
#load data
df = pd.read_csv('data/Privacy_Rights_Clearinghouse_Data_Breaches_Export.csv', 
                 thousands = ',', parse_dates = ['Date Made Public'], encoding = 'utf-8')
df.head()

,Date Made Public,Company,City,State,Type of breach,Type of organization,Total Records,Description of incident,Information Source,Source URL,Year of Breach,Latitude,Longitude
0,2009-10-21,Bullitt County Public Schools,Shepherdsville,Kentucky,DISC,EDU,676.0,A Bullitt County Public Schools \n ...,Dataloss DB,NaN,2009,37.988399,-85.715792
1,2009-10-21,Roane State Community College,Harriman,Tennessee,PORT,EDU,14783.0,Roane State Community College \n ha...,Dataloss DB,NaN,2009,35.933964,-84.552436
2,2009-10-15,Halifax Health,Daytona Beach,Florida,PORT,MED,33000.0,A laptop computer from a Halifax \n ...,Dataloss DB,NaN,2009,29.210815,-81.022833
3,2009-10-04,Suffolk Community College,Selden,New York,DISC,EDU,300.0,Suffolk Community College has \n ag...,Dataloss DB,NaN,2009,40.866487,-73.035663
4,2009-09-28,Penrose Hospital,Colorado Springs,Colorado,PHYS,MED,175.0,Officials at Penrose Hospital bel...,Dataloss DB,NaN,2009,38.833882,-104.821363


### Assessing data

In [3]:
#create a subset exclude data breaches in year 2018
df = df[df['Year of Breach'] != 2018]

In [4]:
# double check number of records of year 2018
sum(df['Year of Breach'] == 2018)

0

In [5]:
# check the basic info. of the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7925 entries, 0 to 8512
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Date Made Public         7925 non-null   datetime64[ns]
 1   Company                  7925 non-null   object        
 2   City                     5492 non-null   object        
 3   State                    7827 non-null   object        
 4   Type of breach           7925 non-null   object        
 5   Type of organization     7925 non-null   object        
 6   Total Records            7925 non-null   float64       
 7   Description of incident  7922 non-null   object        
 8   Information Source       7871 non-null   object        
 9   Source URL               2698 non-null   object        
 10  Year of Breach           7925 non-null   int64         
 11  Latitude                 7925 non-null   float64       
 12  Longitude                7925 non-

- Some columns have missing values
- The data type of 'Total Records' column is float, while it should be int


In [6]:
# check missing values by variables
df.isnull().sum()

Date Made Public              0
Company                       0
City                       2433
State                        98
Type of breach                0
Type of organization          0
Total Records                 0
Description of incident       3
Information Source           54
Source URL                 5227
Year of Breach                0
Latitude                      0
Longitude                     0
dtype: int64

Variable 'City', 'State', 'Total Records', 'Description of incident', 'Information Source' and 'Source URL' have missing values.

In [7]:
# check duplicates
df.duplicated().sum()

13

There are 13 duplicated records

In [8]:
# description of the dataset
df.describe()

,Total Records,Year of Breach,Latitude,Longitude
count,7.925000e+03,7925.000000,7925.000000,7925.000000
mean,1.279903e+06,2012.197603,38.285108,-88.387036
std,4.013869e+07,3.217321,5.337308,19.252870
min,0.000000e+00,2006.000000,-34.603684,-158.056892
25%,0.000000e+00,2010.000000,35.960638,-97.133068
50%,8.620000e+02,2012.000000,39.961176,-82.465144
75%,5.220000e+03,2015.000000,40.760537,-73.978890
max,3.000000e+09,2017.000000,64.837778,139.691706


In [9]:
df['Type of breach'].value_counts()

HACK    2329
PHYS    1661
DISC    1613
PORT    1134
INSD     594
UNKN     287
STAT     239
CARD      68
Name: Type of breach, dtype: int64

In [10]:
df['Type of organization'].value_counts()

MED     3952
BSO     1022
EDU      757
GOV      755
BSF      710
BSR      579
NGO      114
UNKN      36
Name: Type of organization, dtype: int64

Based on the exploration above, the dataset has problems below:
- Variable 'City', 'State', 'Description of incident', 'Information Source' and 'Source URL' have missing values.
- The data type of 'Total Records' column is float, while it should be int
- There are 13 duplicated records
- There are 226 records with value of 'UNKN' in 'Type of breach' column, 36 in column 'Type of organization', we will exclude these.

We will clean the data in the next part

### Cleaning data

#### Filling missing values in 'State' column

In [11]:
# missing values in State
df[df.State.isnull()]

,Date Made Public,Company,City,State,Type of breach,Type of organization,Total Records,Description of incident,Information Source,Source URL,Year of Breach,Latitude,Longitude
373,2007-09-12,"TennCare, Americhoice Inc.",Knoxville,NaN,PORT,MED,67000.0,"There are 67,000 TennCare \n enroll...",Dataloss DB,NaN,2007,35.960638,-83.920739
697,2009-01-27,U.S. Consulate,NaN,NaN,PHYS,GOV,0.0,Hundreds of files - with Social Security numbe...,Media,NaN,2009,37.090240,-95.712891
947,2006-02-23,"Deloitte & Touche, McAfee",NaN,NaN,PORT,BSO,9290.0,"An external auditor lost a CD with names, Soci...",NaN,NaN,2006,37.090240,-95.712891
1112,2009-12-04,Eastern Illinois University,NaN,NaN,HACK,EDU,9000.0,A computer was compromised by a virus. That ca...,NaN,NaN,2009,37.090240,-95.712891
1306,2008-04-17,"SunGard, Connecticut State University System, ...",NaN,NaN,PORT,BSO,0.0,At least 18 colleges are scrambling to inform ...,NaN,NaN,2008,37.090240,-95.712891
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8484,2018-10-22,Byram healthcare,NaN,NaN,UNKN,UNKN,52.0,Information on this security breach is provide...,Indiana Attorney General,https://www.in.gov/attorneygeneral/2874.htm,2016,37.090240,-95.712891
8493,2018-10-16,Chicago property managementand investments,NaN,NaN,UNKN,UNKN,79.0,Information on this security breach is provide...,Indiana Attorney General,https://www.in.gov/attorneygeneral/2874.htm,2017,37.090240,-95.712891
8495,2018-10-12,Buehlers fresh foods llc,NaN,NaN,UNKN,UNKN,2040.0,Information on this security breach is provide...,Indiana Attorney General,https://www.in.gov/attorneygeneral/2874.htm,2016,37.090240,-95.712891
8506,2018-10-08,Trans union llc,NaN,NaN,UNKN,UNKN,242.0,Information on this security breach is provide...,Indiana Attorney General,https://www.in.gov/attorneygeneral/2874.htm,2017,37.090240,-95.712891


From the values of City or Latitude and Longitude, we are able to identify the names of the states and fill in the missing values.

In [12]:
# fill in the missing values of state
df.loc[373, 'State'] = 'TN'
df.loc[697, 'State'] = 'Non-US'
df.loc[947, 'State'] = 'CA'
df.loc[1112, 'State'] = 'IL'
df.loc[1306, 'State'] = 'CT'
df.loc[1767, 'State'] = 'NY'
df.loc[1798, 'State'] = 'KS'
df.loc[1804, 'State'] = 'KS'
df.loc[1853, 'State'] = 'New Hampshire'
df.loc[2173, 'State'] = 'New Jersey'
df.loc[2244, 'State'] = 'CA'
df.loc[2257, 'State'] = 'KS'
df.loc[2259, 'State'] = 'AZ'
df.loc[2266, 'State'] = 'IL'
df.loc[2278, 'State'] = 'AL'
df.loc[2279, 'State'] = 'OH'
df.loc[2284, 'State'] = 'MA'
df.loc[2289, 'State'] = 'NY'
df.loc[2491, 'State'] = 'VA'
df.loc[2501, 'State'] = 'KS'
df.loc[2588, 'State'] = 'GA'
df.loc[2622, 'State'] = 'TX'
df.loc[2623, 'State'] = 'TN'
df.loc[2820, 'State'] = 'MA'
df.loc[2988, 'State'] = 'Germany'
df.loc[3014, 'State'] = 'FL'
df.loc[3081, 'State'] = 'Minnesota'
df.loc[3558, 'State'] = 'Oregon'
df.loc[3321, 'State'] = 'OH'
df.loc[3377, 'State'] = 'CA'
df.loc[3573, 'State'] = 'FL'
df.loc[3580, 'State'] = 'VA'
df.loc[3595, 'State'] = 'NC'
df.loc[3857, 'State'] = 'KS'
df.loc[3866, 'State'] = 'Virginia'
df.loc[3969, 'State'] = 'CA'
df.loc[4055, 'State'] = 'Texas'
df.loc[4079, 'State'] = 'KS'
df.loc[4430, 'State'] = 'CA'
df.loc[4430, 'City'] = 'Los Angeles'
df.loc[4507, 'State'] = 'NY'
df.loc[4577, 'State'] = 'MI'
df.loc[4844, 'State'] = 'PA'
df.loc[4855, 'State'] = 'TX'
df.loc[5364, 'State'] = 'NY'
df.loc[5422, 'State'] = 'PR'
df.loc[5433, 'State'] = 'PR'
df.loc[5443, 'State'] = 'PR'
df.loc[5483, 'State'] = 'PR'
df.loc[5603, 'State'] = 'MA'
df.loc[6148, 'State'] = 'PR'
df.loc[6220, 'State'] = 'PR'
df.loc[6289, 'State'] = 'PR'
df.loc[6289, 'State'] = 'PR'
df.loc[6500, 'State'] = 'PR'
df.loc[7143, 'State'] = 'TX'
df.loc[7527, 'State'] = 'Switzerland'
df.loc[7635, 'State'] = 'Hong Kong'
df.loc[7727, 'State'] = 'CA'
df.loc[7731, 'State'] = 'TX'
df.loc[7732, 'State'] = 'VA'
df.loc[7916, 'State'] = 'MN'
df.loc[7978, 'State'] = 'IL'
df.loc[8178, 'State'] = 'MT'
df.loc[8279, 'State'] = 'TX'
df.loc[8301, 'State'] = 'OK'
df.loc[8405, 'State'] = 'CA'
df.loc[8406, 'State'] = 'CA'
df.loc[8493, 'State'] = 'IL'
df.loc[8183, 'State'] = 'NY'
df.loc[8195, 'State'] = 'IN'
df.loc[8240, 'State'] = 'IL'
df.loc[8245, 'State'] = 'PA'
df.loc[8257, 'State'] = 'OH'
df.loc[8275, 'State'] = 'RI'
df.loc[8281, 'State'] = 'FL'
df.loc[8298, 'State'] = 'CA'
df.loc[8309, 'State'] = 'England'
df.loc[8352, 'State'] = 'MN'
df.loc[8358, 'State'] = 'IL'
df.loc[8363, 'State'] = 'CA'
df.loc[8366, 'State'] = 'VA'
df.loc[8483, 'State'] = 'IL'
df.loc[8506, 'State'] = 'IL'
df.loc[8367, 'State'] = 'FL'
df.loc[8429, 'State'] = 'TX'
df.loc[8432, 'State'] = 'NY'
df.loc[8440, 'State'] = 'FL'
df.loc[8452, 'State'] = 'KY'
df.loc[8495, 'State'] = 'OH'
df.loc[8512, 'State'] = 'CA'
df.loc[8173, 'State'] = 'IN'
df.loc[8300, 'State'] = 'IN'
df.loc[8310, 'State'] = 'IN'
df.loc[8313, 'State'] = 'IN'
df.loc[8351, 'State'] = 'CA'
df.loc[8359, 'State'] = 'CA'
df.loc[8398, 'State'] = 'CA'
df.loc[8456, 'State'] = 'IN'
df.loc[8476, 'State'] = 'IN'
df.loc[8484, 'State'] = 'IN'

In [13]:
# missing values in State after modified
df[df.State.isnull()]

,Date Made Public,Company,City,State,Type of breach,Type of organization,Total Records,Description of incident,Information Source,Source URL,Year of Breach,Latitude,Longitude


In [14]:
df.State.isnull().sum()

0

There are no missing values in 'State' column now.

In [15]:
# total obs by state
df['State'].value_counts()

California     1214
New York        590
Texas           544
Florida         427
Maryland        345
               ... 
AL                1
Switzerland       1
GA                1
Berlin            1
NC                1
Name: State, Length: 94, dtype: int64

In [16]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [17]:
#replace the state abbreviations with full names
df.State.replace(states, inplace = True)

In [18]:
df.State.value_counts()

California       1228
New York          596
Texas             550
Florida           432
Maryland          345
                 ... 
Grand Bahama        1
Beijing             1
Berlin              1
Noord Holland       1
Tokyo               1
Name: State, Length: 70, dtype: int64

#### Droping duplicates

In [19]:
# check the duplicated obs.
df[df.duplicated(keep = False)].sort_values(by = 'Date Made Public', ascending = False)

,Date Made Public,Company,City,State,Type of breach,Type of organization,Total Records,Description of incident,Information Source,Source URL,Year of Breach,Latitude,Longitude
7689,2017-12-08,Mount Carmel Health System,NaN,Ohio,DISC,MED,836.0,Location of breached information: Email\nBusin...,US Department of Health and Human Services,https://ocrportal.hhs.gov/ocr/breach/breach_re...,2017,37.090240,-95.712891
7683,2017-12-08,Mount Carmel Health System,NaN,Ohio,DISC,MED,836.0,Location of breached information: Email\nBusin...,US Department of Health and Human Services,https://ocrportal.hhs.gov/ocr/breach/breach_re...,2017,37.090240,-95.712891
7692,2017-12-05,Oklahoma Department of Human Services,NaN,Oklahoma,HACK,MED,47000.0,Location of breached information: Network Serv...,US Department of Health and Human Services,https://ocrportal.hhs.gov/ocr/breach/breach_re...,2017,37.090240,-95.712891
7691,2017-12-05,Bronson Healthcare Group,NaN,Michigan,HACK,MED,8256.0,Location of breached information: Email\nBusin...,US Department of Health and Human Services,https://ocrportal.hhs.gov/ocr/breach/breach_re...,2017,37.090240,-95.712891
7685,2017-12-05,Oklahoma Department of Human Services,NaN,Oklahoma,HACK,MED,47000.0,Location of breached information: Network Serv...,US Department of Health and Human Services,https://ocrportal.hhs.gov/ocr/breach/breach_re...,2017,37.090240,-95.712891
7684,2017-12-05,Bronson Healthcare Group,NaN,Michigan,HACK,MED,8256.0,Location of breached information: Email\nBusin...,US Department of Health and Human Services,https://ocrportal.hhs.gov/ocr/breach/breach_re...,2017,37.090240,-95.712891
7695,2017-12-01,"CCRM Minneapolis, P.C.",NaN,Minnesota,HACK,MED,3280.0,Location of breached information: Network Serv...,US Department of Health and Human Services,https://ocrportal.hhs.gov/ocr/breach/breach_re...,2017,37.090240,-95.712891
7694,2017-12-01,Washington Health System Greene,NaN,Pennsylvania,PHYS,MED,4145.0,Location of breached information: Other Portab...,US Department of Health and Human Services,https://ocrportal.hhs.gov/ocr/breach/breach_re...,2017,37.090240,-95.712891
7693,2017-12-01,Sinai Health System,NaN,Illinois,HACK,MED,11347.0,Location of breached information: Email\nBusin...,US Department of Health and Human Services,https://ocrportal.hhs.gov/ocr/breach/breach_re...,2017,37.090240,-95.712891
7688,2017-12-01,"CCRM Minneapolis, P.C.",NaN,Minnesota,HACK,MED,3280.0,Location of breached information: Network Serv...,US Department of Health and Human Services,https://ocrportal.hhs.gov/ocr/breach/breach_re...,2017,37.090240,-95.712891


In [20]:
#drop the duplicated obs.
df.drop_duplicates(keep = 'first', inplace = True)

In [21]:
# pd.set_option('max_colwidth', 400)
df[df.State.isnull()]['Description of incident']

Series([], Name: Description of incident, dtype: object)

In [22]:
#missing value after modification
df.isnull().sum()

Date Made Public              0
Company                       0
City                       2422
State                         0
Type of breach                0
Type of organization          0
Total Records                 0
Description of incident       3
Information Source           54
Source URL                 5224
Year of Breach                0
Latitude                      0
Longitude                     0
dtype: int64

In [23]:
df.duplicated().sum()

0

In [24]:
# missing values of 'Source URL' by state after modification 
df[df['Source URL'].isnull()]['State'].value_counts().head(5)

California    894
New York      443
Texas         348
Florida       280
Illinois      203
Name: State, dtype: int64

#### Changing data types of column 'Total Records' 

In [25]:
# Changing data type of column 'Total Records' from float to int
df["Total Records"] = df["Total Records"].astype(int)
df["Total Records"].dtype

dtype('int64')

In [26]:
# value counts of total records
df["Total Records"].value_counts()

0        2187
500        86
1000       78
2000       78
3000       55
         ... 
4469        1
10172       1
1237        1
5335        1
32842       1
Name: Total Records, Length: 2434, dtype: int64

2187 rows have 0 total records. We need to drop that.

#### dropping the '0' values in  'Total Records'

In [27]:
# drop the '0' values in  'Total Records'
df = df[df['Total Records'] != 0]
df["Total Records"].value_counts()

500     86
2000    78
1000    78
3000    55
100     47
        ..
8275     1
3560     1
7666     1
263      1
2123     1
Name: Total Records, Length: 2433, dtype: int64

#### dropping the 'City' column

In [28]:
# drop the 'City' column

df.drop('City', axis=1, inplace=True)
df.head()

,Date Made Public,Company,State,Type of breach,Type of organization,Total Records,Description of incident,Information Source,Source URL,Year of Breach,Latitude,Longitude
0,2009-10-21,Bullitt County Public Schools,Kentucky,DISC,EDU,676,A Bullitt County Public Schools \n ...,Dataloss DB,NaN,2009,37.988399,-85.715792
1,2009-10-21,Roane State Community College,Tennessee,PORT,EDU,14783,Roane State Community College \n ha...,Dataloss DB,NaN,2009,35.933964,-84.552436
2,2009-10-15,Halifax Health,Florida,PORT,MED,33000,A laptop computer from a Halifax \n ...,Dataloss DB,NaN,2009,29.210815,-81.022833
3,2009-10-04,Suffolk Community College,New York,DISC,EDU,300,Suffolk Community College has \n ag...,Dataloss DB,NaN,2009,40.866487,-73.035663
4,2009-09-28,Penrose Hospital,Colorado,PHYS,MED,175,Officials at Penrose Hospital bel...,Dataloss DB,NaN,2009,38.833882,-104.821363


#### exclude the unkowns

In [29]:
# exclude the records with 'UNKN' values
df = df[df['Type of breach'] != 'UNKN']
df['Type of organization'].value_counts()

MED    3365
EDU     568
GOV     523
BSO     392
BSF     327
BSR     255
NGO      68
Name: Type of organization, dtype: int64

### Feature engineering

Based on the current columns, we will create some new columns for the purpose of exploration.

#### breach_type_gA, breach_type_gB, breach_type_gC, breach_type_gD

In [30]:
df['Type of breach'].value_counts()

PHYS    1397
HACK    1388
DISC    1301
PORT     841
INSD     364
STAT     175
CARD      32
Name: Type of breach, dtype: int64

In [31]:
# create a breach type group called breach_type_gA to re-categorize the 
# types of breaches by 'Hacker', 'Others', 'Unknown'
# 'Hacker' correspond to the type of breach 'HACK’ 
# 'Unknown' correspond to the type of breach'Unknown'
# And 'Others' include all the rest types
df['breach_type_gA'] = 'Others'
df.loc[df['Type of breach'] == 'HACK', 'breach_type_gA'] = 'Hacker'
df.loc[df['Type of breach'] == 'UNKN', 'breach_type_gA'] = 'Unknown'

In [32]:
# create breach type group called breach_type_gB to re-categorize the 
# types of breaches by 'Outsider', 'Insider', 'Mixed', and 'Unknown'.
# 'Outsider' includes the original type of breach 'HACK' and 'CARD'
# 'Insider' includes the original type of breach 'INSD' and 'DISC'
# 'Unknown' right as the original type of breach 'Unknown' 
# 'Mixed' includes all the rest types of breach for which we could not say for
#  sure whether it is outsider or insider since it depends on each specific event
df['breach_type_gB'] = 'Mixed'
df.loc[df['Type of breach'] == 'HACK', 'breach_type_gB'] = 'Outsider'
df.loc[df['Type of breach'] == 'CARD', 'breach_type_gB'] = 'Outsider'
df.loc[df['Type of breach'] == 'INSD', 'breach_type_gB'] = 'Insider'
df.loc[df['Type of breach'] == 'DISC', 'breach_type_gB'] = 'Insider'
df.loc[df['Type of breach'] == 'UNKN', 'breach_type_gB'] = 'Unknown'

In [33]:
# created a breach type group called 'breach_type_gC' to re-categorize the 
# types of breaches by 'Online', 'Offline', 'Others', 'Unknown', 
# 'Online' includes the type of breach 'CARD’ and'HACK',
# 'Offline' includes the type of breach 'PHYS' and 'PORT'. 
# 'Unknown' correspond to the type of breach'UNKN'. 
# And 'Others' include all the rest types which cover both online and offline

df['breach_type_gC'] = 'Others'
df.loc[df['Type of breach'].isin(['PHYS','PORT']), 'breach_type_gC'] = 'Offline'
df.loc[df['Type of breach'].isin(['CARD', 'HACK']), 'breach_type_gC'] = 'Online'
df.loc[df['Type of breach'] == 'UNKN', 'breach_type_gC'] = 'Unknown'

In [34]:
# created a breach type group called 'breach_type_gD' to re-categorize the 
# types of breaches by 'Network', 'Local', 'Others', 'Unknown', 
# 'Network' includes the type of breach 'CARD’ and'HACK',
# 'Local' includes the type of breach 'PHYS', 'PORT' and 'STAT'. 
# 'Unknown' correspond to the type of breach'UNKN'. 
# And 'Others' include all the rest types which cover both network and local

df['breach_type_gD'] = 'Others'
df.loc[df['Type of breach'].isin(['PHYS','PORT', 'STAT']), 'breach_type_gD'] = 'Local'
df.loc[df['Type of breach'].isin(['CARD', 'HACK', 'DISC']), 'breach_type_gD'] = 'Network'

In [35]:
# check the new columns
df.head()

,Date Made Public,Company,State,Type of breach,Type of organization,Total Records,Description of incident,Information Source,Source URL,Year of Breach,Latitude,Longitude,breach_type_gA,breach_type_gB,breach_type_gC,breach_type_gD
0,2009-10-21,Bullitt County Public Schools,Kentucky,DISC,EDU,676,A Bullitt County Public Schools \n ...,Dataloss DB,NaN,2009,37.988399,-85.715792,Others,Insider,Others,Network
1,2009-10-21,Roane State Community College,Tennessee,PORT,EDU,14783,Roane State Community College \n ha...,Dataloss DB,NaN,2009,35.933964,-84.552436,Others,Mixed,Offline,Local
2,2009-10-15,Halifax Health,Florida,PORT,MED,33000,A laptop computer from a Halifax \n ...,Dataloss DB,NaN,2009,29.210815,-81.022833,Others,Mixed,Offline,Local
3,2009-10-04,Suffolk Community College,New York,DISC,EDU,300,Suffolk Community College has \n ag...,Dataloss DB,NaN,2009,40.866487,-73.035663,Others,Insider,Others,Network
4,2009-09-28,Penrose Hospital,Colorado,PHYS,MED,175,Officials at Penrose Hospital bel...,Dataloss DB,NaN,2009,38.833882,-104.821363,Others,Mixed,Offline,Local


In [36]:
# value counts of each category in 'breach_type_gA'
df['breach_type_gA'].value_counts()

Others    4110
Hacker    1388
Name: breach_type_gA, dtype: int64

In [37]:
# value counts of each category in 'breach_type_gB'
df['breach_type_gB'].value_counts()

Mixed       2413
Insider     1665
Outsider    1420
Name: breach_type_gB, dtype: int64

In [38]:
# value counts of each category in 'breach_type_gC'
df['breach_type_gC'].value_counts()

Offline    2238
Others     1840
Online     1420
Name: breach_type_gC, dtype: int64

In [39]:
# value counts of each category in 'breach_type_gD'
df['breach_type_gD'].value_counts()

Network    2721
Local      2413
Others      364
Name: breach_type_gD, dtype: int64

#### region-A and region-B

In [40]:
# create a new variable 'region-A' from ‘State’,with the values‘Northeast’, ‘Midwest’,
#‘South’, ‘West’ and 'Non_US' for locations in other countries rather than US.
NE = [ 'Maine', 'New York', 'New Jersey', 'Vermont', 'Massachusetts', 
    'Rhode Island','Connecticut', 'New Hampshire',
             'Pennsylvania']
MW = ['Illinois', 'Indiana', 'Iowa', 'Kansas', 'Michigan', 'Minnesota', 
           'Missouri', 'Nebraska', 'North Dakota', 'Ohio', 'South Dakota', 'Wisconsin']
S = ['Delaware', 'Florida', 'Georgia', 'Maryland', 'North Carolina', 'South Carolina',
         'Virginia', 'District of Columbia', 'West Virginia', 'Alabama', 'Kentucky', 
         'Mississippi', 'Tennessee', 'Arkansas', 'Louisiana', 'Oklahoma', 'Texas']
W = ['Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'New Mexico', 'Utah', 'Wyoming', 
        'Alaska', 'California', 'Hawaii', 'Oregon', 'Washington']
df['region_A'] = 'Non_US'
df.loc[df.State.isin(NE), 'region_A'] = 'Northeast'
df.loc[df.State.isin(MW), 'region_A'] = 'Midwest'
df.loc[df.State.isin(S), 'region_A'] = 'South'
df.loc[df.State.isin(W), 'region_A'] = 'West'

In [41]:
# number of states in each region in US
len(NE), len(MW), len(S), len(W)

(9, 12, 17, 13)

In [42]:
# create a new variable 'region-B' from 'State',with the values‘Northeast’, ‘Midwest’,
#‘Southeast’, 'Southwest', ‘West’ and 'Non_US' for locations in other countries rather than US.
Northeast = [ 'Maine', 'New York', 'New Jersey', 'Vermont', 'Massachusetts', 
    'Rhode Island','Connecticut', 'New Hampshire',
             'Pennsylvania', 'Delaware', 'Maryland']
Midwest = ['Illinois', 'Indiana', 'Iowa', 'Kansas', 'Michigan', 'Minnesota', 
           'Missouri', 'Nebraska', 'North Dakota', 'Ohio', 'South Dakota', 'Wisconsin']
Southwest = ['Texas', 'Oklahoma', 'New Mexico', 'Arizona']
Southeast = ['Florida', 'Georgia', 'North Carolina', 'South Carolina',
         'Virginia', 'West Virginia', 'Alabama', 'Kentucky', 
         'Mississippi', 'Tennessee', 'Arkansas', 'Louisiana']
West = ['Colorado', 'Idaho', 'Montana', 'Nevada', 'Utah', 'Wyoming', 
        'Alaska', 'California', 'Hawaii', 'Oregon', 'Washington']
df['region_B'] = 'Non_US'
df.loc[df.State.isin(Northeast), 'region_B'] = 'Northeast'
df.loc[df.State.isin(Midwest), 'region_B'] = 'Midwest'
df.loc[df.State.isin(Southwest), 'region_B'] = 'Southwest'
df.loc[df.State.isin(Southeast), 'region_B'] = 'Southeast'
df.loc[df.State.isin(West), 'region_B'] = 'West'

In [43]:
# number of states in each region in US
len(Northeast), len(Midwest),len(Southwest), len(Southeast), len(West)

(11, 12, 4, 12, 11)

In [44]:
df.head()

,Date Made Public,Company,State,Type of breach,Type of organization,Total Records,Description of incident,Information Source,Source URL,Year of Breach,Latitude,Longitude,breach_type_gA,breach_type_gB,breach_type_gC,breach_type_gD,region_A,region_B
0,2009-10-21,Bullitt County Public Schools,Kentucky,DISC,EDU,676,A Bullitt County Public Schools \n ...,Dataloss DB,NaN,2009,37.988399,-85.715792,Others,Insider,Others,Network,South,Southeast
1,2009-10-21,Roane State Community College,Tennessee,PORT,EDU,14783,Roane State Community College \n ha...,Dataloss DB,NaN,2009,35.933964,-84.552436,Others,Mixed,Offline,Local,South,Southeast
2,2009-10-15,Halifax Health,Florida,PORT,MED,33000,A laptop computer from a Halifax \n ...,Dataloss DB,NaN,2009,29.210815,-81.022833,Others,Mixed,Offline,Local,South,Southeast
3,2009-10-04,Suffolk Community College,New York,DISC,EDU,300,Suffolk Community College has \n ag...,Dataloss DB,NaN,2009,40.866487,-73.035663,Others,Insider,Others,Network,Northeast,Northeast
4,2009-09-28,Penrose Hospital,Colorado,PHYS,MED,175,Officials at Penrose Hospital bel...,Dataloss DB,NaN,2009,38.833882,-104.821363,Others,Mixed,Offline,Local,West,West


In [45]:
# value counts
df.region_A.value_counts()

South        1956
West         1283
Midwest      1086
Northeast    1030
Non_US        143
Name: region_A, dtype: int64

In [46]:
# value counts
df.region_B.value_counts()

Northeast    1315
Southeast    1226
West         1149
Midwest      1086
Southwest     579
Non_US        143
Name: region_B, dtype: int64

#### Age of Law

In [47]:
# create a new variable called 'Age of Law' to describe how many years a state have had a law 
# regarding the data breach
dict_law = {'California':16, 'Oregon':12, 'Wisconsin':13, 'Florida':14, 'Maryland':11, 
            'Texas':10, 'Virginia':11, 'Georgia':14, 'New Jersey':13, 'New York':14,
            'Massachusetts':12, 'Indiana':13, 'Washington':14, 'Minnesota':13, 'Nevada':14,
            'Pennsylvania':13, 'North Carolina':14, 'South Carolina':10, 'Nebraska':13, 
            'Kansas':13, 'Illinois':13, 'Missouri':10, 'Tennessee':14, 'Maine':13, 'Arizona':13, 
            'Colorado':13, 'Delaware':14, 'Utah':12, 'Connecticut':13,'Ohio':13, 'Kentucky':0, 
            'New Hampshire':12, 'Oklahoma':13, 'Arkansas':14, 'Iowa':11, 'Michigan':12, 'New Mexico':0,
            'Alaska':10, 'North Dakota':14, 'Louisiana':13, 'Idaho':13, 'Vermont':12, 'Rhode Island':13, 
            'Mississippi':8, 'Hawaii':12, 'Alabama':0, 'West Virginia':11, 'Montana':13, 'Wyoming':12,
            'South Dakota':0, 'District Of Columbia':12, 'District Of Columbia':0}
df['age_of_Law'] = df['State'].replace(dict_law, inplace = False)
df.head()

,Date Made Public,Company,State,Type of breach,Type of organization,Total Records,Description of incident,Information Source,Source URL,Year of Breach,Latitude,Longitude,breach_type_gA,breach_type_gB,breach_type_gC,breach_type_gD,region_A,region_B,age_of_Law
0,2009-10-21,Bullitt County Public Schools,Kentucky,DISC,EDU,676,A Bullitt County Public Schools \n ...,Dataloss DB,NaN,2009,37.988399,-85.715792,Others,Insider,Others,Network,South,Southeast,0
1,2009-10-21,Roane State Community College,Tennessee,PORT,EDU,14783,Roane State Community College \n ha...,Dataloss DB,NaN,2009,35.933964,-84.552436,Others,Mixed,Offline,Local,South,Southeast,14
2,2009-10-15,Halifax Health,Florida,PORT,MED,33000,A laptop computer from a Halifax \n ...,Dataloss DB,NaN,2009,29.210815,-81.022833,Others,Mixed,Offline,Local,South,Southeast,14
3,2009-10-04,Suffolk Community College,New York,DISC,EDU,300,Suffolk Community College has \n ag...,Dataloss DB,NaN,2009,40.866487,-73.035663,Others,Insider,Others,Network,Northeast,Northeast,14
4,2009-09-28,Penrose Hospital,Colorado,PHYS,MED,175,Officials at Penrose Hospital bel...,Dataloss DB,NaN,2009,38.833882,-104.821363,Others,Mixed,Offline,Local,West,West,13


In [48]:
# value counts
df.age_of_Law.value_counts()

13                  1586
14                  1391
16                   657
10                   573
11                   469
12                   457
0                    290
Puerto Rico           38
8                     26
Quebec                 2
Ontario                2
Guangdong              1
Beijing                1
British Columbia       1
London                 1
Noord Holland          1
Berlin                 1
Germany                1
Name: age_of_Law, dtype: int64

#### Org_group

In [49]:
# create a new variable 'Org_group', assign value 'F_regulated' to 'Org_group' where the type of organization is 
# 'EDU'/'MED'/'BSF', for other types of organization assign value 'non_regulated'
df['org_group'] = 'non_F_regulated'
regulated_org = ['EDU', 'MED', 'BSF']
df.loc[df['Type of organization'].isin(regulated_org), 'org_group'] = 'F_regulated'

#### records_group

In [50]:
df['Total Records'].value_counts()

2000      77
1000      75
500       74
3000      54
5000      45
          ..
846        1
2036       1
177000     1
327        1
2123       1
Name: Total Records, Length: 2363, dtype: int64

In [51]:
# there are 3 incidents with the total number of record of more than 1 billion,
# we categorize those 3 obs. as extraordinary, other obs. wil be noted as Others 
df.sort_values('Total Records', ascending = False)

,Date Made Public,Company,State,Type of breach,Type of organization,Total Records,Description of incident,Information Source,Source URL,Year of Breach,Latitude,Longitude,breach_type_gA,breach_type_gB,breach_type_gC,breach_type_gD,region_A,region_B,age_of_Law,org_group
5066,2016-12-14,Yahoo,California,HACK,BSO,3000000000,"""Yahoo Inc (YHOO.O) warned on Wednesday that i...",Media,NaN,2016,37.368830,-122.036350,Hacker,Outsider,Online,Network,West,West,16,non_F_regulated
5171,2017-03-08,River City Media,Oregon,DISC,BSO,1370000000,"""One of the world's allegedly most prolific sp...",Media,NaN,2017,45.520842,-122.680344,Others,Insider,Others,Network,West,West,12,non_F_regulated
4207,2014-08-05,Russian hacking discovered by Hold Security,Wisconsin,HACK,BSO,1000000000,"""A gang of Russian hackers has amassed over 1 ...",Media,NaN,2014,41.899183,-87.946671,Hacker,Outsider,Online,Network,Midwest,Midwest,13,non_F_regulated
4944,2016-09-22,Yahoo,California,HACK,BSO,500000000,"""Yahoo is poised to confirm a massive data bre...",Media,NaN,2016,37.368830,-122.036350,Hacker,Outsider,Online,Network,West,West,16,non_F_regulated
5029,2016-11-16,FriendFinder,California,HACK,BSO,412000000,"""A hack against popular adult dating and enter...",Media,NaN,2016,37.407315,-122.017311,Hacker,Outsider,Online,Network,West,West,16,non_F_regulated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7231,2014-09-23,"Six Continents Hotels, Inc.",Maryland,INSD,MED,1,"name, payment card info\nLocation of breached ...",Maryland Attorney General,http://www.marylandattorneygeneral.gov/Pages/I...,2014,40.760537,-73.978890,Others,Insider,Others,Others,South,Northeast,11,F_regulated
7354,2014-04-07,FujiFilm,New York,HACK,MED,1,email system accessed without authorization\nL...,Maryland Attorney General,http://www.marylandattorneygeneral.gov/Pages/I...,2014,39.501681,-76.979392,Hacker,Outsider,Online,Network,Northeast,Northeast,14,F_regulated
7787,2018-01-10,St. Vincent Warrick,Indiana,DISC,MED,1,St. Vincent learned of an inadvertent disclosu...,NaN,NaN,2017,38.973639,-92.743242,Others,Insider,Others,Network,Midwest,Midwest,13,F_regulated
7235,2014-09-18,"CareCentrix, Inc.",Maryland,INSD,MED,1,"name, address, dob, ssn, health plan numbers\n...",Maryland Attorney General,http://www.marylandattorneygeneral.gov/Pages/I...,2014,40.760537,-73.978890,Others,Insider,Others,Others,South,Northeast,11,F_regulated


In [52]:
# create variable 'records_group'
df['records_group'] = 'Others'
df.loc[5066, 'records_group'] = 'Extraordinary'
df.loc[5171, 'records_group'] = 'Extraordinary'
df.loc[4207, 'records_group'] = 'Extraordinary'
df.head()

,Date Made Public,Company,State,Type of breach,Type of organization,Total Records,Description of incident,Information Source,Source URL,Year of Breach,...,Longitude,breach_type_gA,breach_type_gB,breach_type_gC,breach_type_gD,region_A,region_B,age_of_Law,org_group,records_group
0,2009-10-21,Bullitt County Public Schools,Kentucky,DISC,EDU,676,A Bullitt County Public Schools \n ...,Dataloss DB,NaN,2009,...,-85.715792,Others,Insider,Others,Network,South,Southeast,0,F_regulated,Others
1,2009-10-21,Roane State Community College,Tennessee,PORT,EDU,14783,Roane State Community College \n ha...,Dataloss DB,NaN,2009,...,-84.552436,Others,Mixed,Offline,Local,South,Southeast,14,F_regulated,Others
2,2009-10-15,Halifax Health,Florida,PORT,MED,33000,A laptop computer from a Halifax \n ...,Dataloss DB,NaN,2009,...,-81.022833,Others,Mixed,Offline,Local,South,Southeast,14,F_regulated,Others
3,2009-10-04,Suffolk Community College,New York,DISC,EDU,300,Suffolk Community College has \n ag...,Dataloss DB,NaN,2009,...,-73.035663,Others,Insider,Others,Network,Northeast,Northeast,14,F_regulated,Others
4,2009-09-28,Penrose Hospital,Colorado,PHYS,MED,175,Officials at Penrose Hospital bel...,Dataloss DB,NaN,2009,...,-104.821363,Others,Mixed,Offline,Local,West,West,13,F_regulated,Others


In [53]:
df.records_group.value_counts()

Others           5495
Extraordinary       3
Name: records_group, dtype: int64

In [54]:
df[df.records_group != 'Others']

,Date Made Public,Company,State,Type of breach,Type of organization,Total Records,Description of incident,Information Source,Source URL,Year of Breach,...,Longitude,breach_type_gA,breach_type_gB,breach_type_gC,breach_type_gD,region_A,region_B,age_of_Law,org_group,records_group
4207,2014-08-05,Russian hacking discovered by Hold Security,Wisconsin,HACK,BSO,1000000000,"""A gang of Russian hackers has amassed over 1 ...",Media,NaN,2014,...,-87.946671,Hacker,Outsider,Online,Network,Midwest,Midwest,13,non_F_regulated,Extraordinary
5066,2016-12-14,Yahoo,California,HACK,BSO,3000000000,"""Yahoo Inc (YHOO.O) warned on Wednesday that i...",Media,NaN,2016,...,-122.036350,Hacker,Outsider,Online,Network,West,West,16,non_F_regulated,Extraordinary
5171,2017-03-08,River City Media,Oregon,DISC,BSO,1370000000,"""One of the world's allegedly most prolific sp...",Media,NaN,2017,...,-122.680344,Others,Insider,Others,Network,West,West,12,non_F_regulated,Extraordinary


In [55]:
# description of 'Total Records' for the 'Others' group
df[df.records_group == 'Others']['Total Records'].describe()

count    5.495000e+03
mean     8.315699e+05
std      1.268289e+07
min      1.000000e+00
25%      6.580000e+02
50%      2.088000e+03
75%      1.050050e+04
max      5.000000e+08
Name: Total Records, dtype: float64

#### records_index

In [56]:
# get the mean of Total Records for records_group Extraordinary and Others.
extra_mean = df.groupby('records_group')['Total Records'].mean()['Extraordinary']
others_mean = df.groupby('records_group')['Total Records'].mean()['Others']

In [57]:
# Create a new variable named records_index, the values of records_index would be 
# total records divided by mean
df['records_index'] = df['Total Records'] / others_mean
df.loc[4207, 'records_index'] = (df['Total Records'][4207] / extra_mean)
df.loc[5066, 'records_index'] = (df['Total Records'][5066] / extra_mean)
df.loc[5171, 'records_index'] = (df['Total Records'][5171] / extra_mean)

#### severity_index1, severity_index2, severity_index3

In [58]:
# adjust to width to show all the columns
pd.set_option('display.max_columns', None)
df.head()

,Date Made Public,Company,State,Type of breach,Type of organization,Total Records,Description of incident,Information Source,Source URL,Year of Breach,Latitude,Longitude,breach_type_gA,breach_type_gB,breach_type_gC,breach_type_gD,region_A,region_B,age_of_Law,org_group,records_group,records_index
0,2009-10-21,Bullitt County Public Schools,Kentucky,DISC,EDU,676,A Bullitt County Public Schools \n ...,Dataloss DB,NaN,2009,37.988399,-85.715792,Others,Insider,Others,Network,South,Southeast,0,F_regulated,Others,0.000813
1,2009-10-21,Roane State Community College,Tennessee,PORT,EDU,14783,Roane State Community College \n ha...,Dataloss DB,NaN,2009,35.933964,-84.552436,Others,Mixed,Offline,Local,South,Southeast,14,F_regulated,Others,0.017777
2,2009-10-15,Halifax Health,Florida,PORT,MED,33000,A laptop computer from a Halifax \n ...,Dataloss DB,NaN,2009,29.210815,-81.022833,Others,Mixed,Offline,Local,South,Southeast,14,F_regulated,Others,0.039684
3,2009-10-04,Suffolk Community College,New York,DISC,EDU,300,Suffolk Community College has \n ag...,Dataloss DB,NaN,2009,40.866487,-73.035663,Others,Insider,Others,Network,Northeast,Northeast,14,F_regulated,Others,0.000361
4,2009-09-28,Penrose Hospital,Colorado,PHYS,MED,175,Officials at Penrose Hospital bel...,Dataloss DB,NaN,2009,38.833882,-104.821363,Others,Mixed,Offline,Local,West,West,13,F_regulated,Others,0.000210


In [59]:
# use a sigmoid function to turn the total records into scale between 0 and 10
df['records_scale'] = 10/(1 + np.exp(-df['Total Records'])) 
df['records_scale'].value_counts()

10.000000    5170
7.310586       43
10.000000      22
8.807971       21
9.999939       16
10.000000      16
9.999999       14
9.999833       14
9.999997       12
9.999546       12
9.999977       11
10.000000      11
9.525741       10
10.000000       9
9.933071        9
9.820138        9
9.996646        8
10.000000       8
10.000000       8
10.000000       7
10.000000       6
9.999992        6
10.000000       6
9.975274        5
10.000000       5
10.000000       4
10.000000       4
10.000000       4
9.990889        4
10.000000       4
10.000000       4
10.000000       3
9.998766        3
10.000000       3
10.000000       3
10.000000       2
10.000000       2
Name: records_scale, dtype: int64

In [60]:
# create a new variable type_scale to meature the cost by breach type
type_wt_dict = {'DISC': 1, 'PORT': 7, 'PHYS': 7, 'UNKN': 5, 'INSD': 7, 'HACK': 10, 'STAT': 7, 'CARD': 10}
df['type_scale'] = df['Type of breach'].map(type_wt_dict)
df.head(5)

,Date Made Public,Company,State,Type of breach,Type of organization,Total Records,Description of incident,Information Source,Source URL,Year of Breach,Latitude,Longitude,breach_type_gA,breach_type_gB,breach_type_gC,breach_type_gD,region_A,region_B,age_of_Law,org_group,records_group,records_index,records_scale,type_scale
0,2009-10-21,Bullitt County Public Schools,Kentucky,DISC,EDU,676,A Bullitt County Public Schools \n ...,Dataloss DB,NaN,2009,37.988399,-85.715792,Others,Insider,Others,Network,South,Southeast,0,F_regulated,Others,0.000813,10.0,1
1,2009-10-21,Roane State Community College,Tennessee,PORT,EDU,14783,Roane State Community College \n ha...,Dataloss DB,NaN,2009,35.933964,-84.552436,Others,Mixed,Offline,Local,South,Southeast,14,F_regulated,Others,0.017777,10.0,7
2,2009-10-15,Halifax Health,Florida,PORT,MED,33000,A laptop computer from a Halifax \n ...,Dataloss DB,NaN,2009,29.210815,-81.022833,Others,Mixed,Offline,Local,South,Southeast,14,F_regulated,Others,0.039684,10.0,7
3,2009-10-04,Suffolk Community College,New York,DISC,EDU,300,Suffolk Community College has \n ag...,Dataloss DB,NaN,2009,40.866487,-73.035663,Others,Insider,Others,Network,Northeast,Northeast,14,F_regulated,Others,0.000361,10.0,1
4,2009-09-28,Penrose Hospital,Colorado,PHYS,MED,175,Officials at Penrose Hospital bel...,Dataloss DB,NaN,2009,38.833882,-104.821363,Others,Mixed,Offline,Local,West,West,13,F_regulated,Others,0.000210,10.0,7


In [61]:
# create a new variable org_scale to meature the cost by organization
org_wt_dict = {'EDU': 8, 'MED': 10, 'BSF': 9, 'GOV': 1, 'BSO': 6, 'BSR': 3, 'NGO': 1, 'UNKN': 5}
df['org_scale'] = df['Type of organization'].map(org_wt_dict)
df.head(5)

,Date Made Public,Company,State,Type of breach,Type of organization,Total Records,Description of incident,Information Source,Source URL,Year of Breach,Latitude,Longitude,breach_type_gA,breach_type_gB,breach_type_gC,breach_type_gD,region_A,region_B,age_of_Law,org_group,records_group,records_index,records_scale,type_scale,org_scale
0,2009-10-21,Bullitt County Public Schools,Kentucky,DISC,EDU,676,A Bullitt County Public Schools \n ...,Dataloss DB,NaN,2009,37.988399,-85.715792,Others,Insider,Others,Network,South,Southeast,0,F_regulated,Others,0.000813,10.0,1,8
1,2009-10-21,Roane State Community College,Tennessee,PORT,EDU,14783,Roane State Community College \n ha...,Dataloss DB,NaN,2009,35.933964,-84.552436,Others,Mixed,Offline,Local,South,Southeast,14,F_regulated,Others,0.017777,10.0,7,8
2,2009-10-15,Halifax Health,Florida,PORT,MED,33000,A laptop computer from a Halifax \n ...,Dataloss DB,NaN,2009,29.210815,-81.022833,Others,Mixed,Offline,Local,South,Southeast,14,F_regulated,Others,0.039684,10.0,7,10
3,2009-10-04,Suffolk Community College,New York,DISC,EDU,300,Suffolk Community College has \n ag...,Dataloss DB,NaN,2009,40.866487,-73.035663,Others,Insider,Others,Network,Northeast,Northeast,14,F_regulated,Others,0.000361,10.0,1,8
4,2009-09-28,Penrose Hospital,Colorado,PHYS,MED,175,Officials at Penrose Hospital bel...,Dataloss DB,NaN,2009,38.833882,-104.821363,Others,Mixed,Offline,Local,West,West,13,F_regulated,Others,0.000210,10.0,7,10


In [62]:
# create severity_index1, severity_index2 and severity_index3
df['severity_index1'] = df['records_scale'] + df['type_scale']
df['severity_index2'] = df['records_scale'] + df['org_scale']
df['severity_index3'] = df['records_scale'] + df['org_scale'] + df['type_scale']
df.head()

,Date Made Public,Company,State,Type of breach,Type of organization,Total Records,Description of incident,Information Source,Source URL,Year of Breach,Latitude,Longitude,breach_type_gA,breach_type_gB,breach_type_gC,breach_type_gD,region_A,region_B,age_of_Law,org_group,records_group,records_index,records_scale,type_scale,org_scale,severity_index1,severity_index2,severity_index3
0,2009-10-21,Bullitt County Public Schools,Kentucky,DISC,EDU,676,A Bullitt County Public Schools \n ...,Dataloss DB,NaN,2009,37.988399,-85.715792,Others,Insider,Others,Network,South,Southeast,0,F_regulated,Others,0.000813,10.0,1,8,11.0,18.0,19.0
1,2009-10-21,Roane State Community College,Tennessee,PORT,EDU,14783,Roane State Community College \n ha...,Dataloss DB,NaN,2009,35.933964,-84.552436,Others,Mixed,Offline,Local,South,Southeast,14,F_regulated,Others,0.017777,10.0,7,8,17.0,18.0,25.0
2,2009-10-15,Halifax Health,Florida,PORT,MED,33000,A laptop computer from a Halifax \n ...,Dataloss DB,NaN,2009,29.210815,-81.022833,Others,Mixed,Offline,Local,South,Southeast,14,F_regulated,Others,0.039684,10.0,7,10,17.0,20.0,27.0
3,2009-10-04,Suffolk Community College,New York,DISC,EDU,300,Suffolk Community College has \n ag...,Dataloss DB,NaN,2009,40.866487,-73.035663,Others,Insider,Others,Network,Northeast,Northeast,14,F_regulated,Others,0.000361,10.0,1,8,11.0,18.0,19.0
4,2009-09-28,Penrose Hospital,Colorado,PHYS,MED,175,Officials at Penrose Hospital bel...,Dataloss DB,NaN,2009,38.833882,-104.821363,Others,Mixed,Offline,Local,West,West,13,F_regulated,Others,0.000210,10.0,7,10,17.0,20.0,27.0


#### severity_index4, severity_index5

In [63]:
# create severity_index4 which is the mean of the three scales
df['severity_index4'] = df[['records_scale','org_scale', 'type_scale']].mean(axis=1)

In [64]:
# create severity_index5 which is the variance of the three scales
df['severity_index5'] = df[['records_scale','org_scale', 'type_scale']].var(axis=1)
df.head()

,Date Made Public,Company,State,Type of breach,Type of organization,Total Records,Description of incident,Information Source,Source URL,Year of Breach,Latitude,Longitude,breach_type_gA,breach_type_gB,breach_type_gC,breach_type_gD,region_A,region_B,age_of_Law,org_group,records_group,records_index,records_scale,type_scale,org_scale,severity_index1,severity_index2,severity_index3,severity_index4,severity_index5
0,2009-10-21,Bullitt County Public Schools,Kentucky,DISC,EDU,676,A Bullitt County Public Schools \n ...,Dataloss DB,NaN,2009,37.988399,-85.715792,Others,Insider,Others,Network,South,Southeast,0,F_regulated,Others,0.000813,10.0,1,8,11.0,18.0,19.0,6.333333,22.333333
1,2009-10-21,Roane State Community College,Tennessee,PORT,EDU,14783,Roane State Community College \n ha...,Dataloss DB,NaN,2009,35.933964,-84.552436,Others,Mixed,Offline,Local,South,Southeast,14,F_regulated,Others,0.017777,10.0,7,8,17.0,18.0,25.0,8.333333,2.333333
2,2009-10-15,Halifax Health,Florida,PORT,MED,33000,A laptop computer from a Halifax \n ...,Dataloss DB,NaN,2009,29.210815,-81.022833,Others,Mixed,Offline,Local,South,Southeast,14,F_regulated,Others,0.039684,10.0,7,10,17.0,20.0,27.0,9.000000,3.000000
3,2009-10-04,Suffolk Community College,New York,DISC,EDU,300,Suffolk Community College has \n ag...,Dataloss DB,NaN,2009,40.866487,-73.035663,Others,Insider,Others,Network,Northeast,Northeast,14,F_regulated,Others,0.000361,10.0,1,8,11.0,18.0,19.0,6.333333,22.333333
4,2009-09-28,Penrose Hospital,Colorado,PHYS,MED,175,Officials at Penrose Hospital bel...,Dataloss DB,NaN,2009,38.833882,-104.821363,Others,Mixed,Offline,Local,West,West,13,F_regulated,Others,0.000210,10.0,7,10,17.0,20.0,27.0,9.000000,3.000000


### Saving data

In [65]:
#save data to csv file 
df.to_csv('data/data_breach.csv',index=False)